In [1]:
import pandas as pd
from typing import List, cast, Literal
from sklearn.multioutput import RegressorChain
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

In [2]:
stores = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/stores.csv")
holidays_events = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv",dtype={
            "type": "category",
            "locale": "category",
            "locale_name": "category",
            "description": "category",
            "transferred": "bool",
            "date": "period[D]"
        }
)
store_sales = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/train.csv",
        dtype={
            "store_nbr": "category",
            "family": "category",
            "sales": "float32",
            "onpromotion": "uint32",
            "date": "period[D]"
        }
)
query = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/test.csv",
        dtype={
            "store_nbr": "category",
            "family": "category",
            "onpromotion": "uint32",
            "date": "period[D]"
        }
)
oil = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/oil.csv", 
        dtype={            
            "date": "period[D]"
        }
)
transactions = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/transactions.csv")

In [3]:
main_index: List[str] = [
    "date",
    "store_nbr",
    "family",
]
secondary_index: List[str] = ["store_nbr", "family"]
unique_families = store_sales["family"].unique()
unique_stores = store_sales["store_nbr"].unique()

In [4]:
holidays_to_consider = holidays_events[
        (holidays_events["transferred"].eq(False))
        & holidays_events["locale"].isin(["National"])
    ].drop_duplicates(keep="first", subset=["date"])

In [5]:
 
all_periods = pd.period_range("2013-01-01", "2017-08-31")
oil_prices = oil.set_index("date").reindex(all_periods).rename_axis("date").ffill().bfill()
oil_prices

,dcoilwtico
date,
2013-01-01,93.14
2013-01-02,93.14
2013-01-03,92.97
2013-01-04,93.12
2013-01-05,93.12
...,...
2017-08-27,47.65
2017-08-28,46.40
2017-08-29,46.46


In [6]:
data = pd.concat([store_sales.set_index(main_index), query.set_index(main_index)],axis=0)
data_extended_1 = data.reset_index(secondary_index).join(oil_prices).set_index(secondary_index,append=True).rename(columns={"dcoilwtico": "oil"})
display(data_extended_1)
data_extended_1["is_holiday"] = data_extended_1.index.get_level_values("date").isin(holidays_to_consider["date"])
data_extended_1.loc[:,["year"]] = data_extended_1.index.get_level_values("date").year
data_extended_1.loc[:,["month"]] = data_extended_1.index.get_level_values("date").month
data_extended_1.loc[:,["day_of_week"]] = data_extended_1.index.get_level_values("date").day_of_week

data_extended_1 = data_extended_1.astype({
    "year":"category",
    "month":"category",
    "day_of_week":"category"
})


data_final = data_extended_1.copy()
mdata = data_extended_1.copy()
display(data_final)

id  sales  onpromotion  \
date       store_nbr family                                                    
2013-01-01 1         AUTOMOTIVE                        0    0.0            0   
                     BABY CARE                         1    0.0            0   
                     BEAUTY                            2    0.0            0   
                     BEVERAGES                         3    0.0            0   
                     BOOKS                             4    0.0            0   
...                                                  ...    ...          ...   
2017-08-31 9         POULTRY                     3029395    NaN            1   
                     PREPARED FOODS              3029396    NaN            0   
                     PRODUCE                     3029397    NaN            1   
                     SCHOOL AND OFFICE SUPPLIES  3029398    NaN            9   
                     SEAFOOD                     3029399    NaN            0   

                                                   oil  
date       store_nbr family                             
2013-01-01 1         AUTOMOTIVE                  93.14  
                     BABY CARE                   93.14  
                     BEAUTY                      93.14  
                     BEVERAGES                   93.14  
                     BOOKS                       93.14  
...                                                ...  
2017-08-31 9         POULTRY                     47.26  
                     PREPARED FOODS              47.26  
                     PRODUCE                     47.26  
                     SCHOOL AND OFFICE SUPPLIES  47.26  
                     SEAFOOD                     47.26  

[3029400 rows x 4 columns]

id  sales  onpromotion  \
date       store_nbr family                                                    
2013-01-01 1         AUTOMOTIVE                        0    0.0            0   
                     BABY CARE                         1    0.0            0   
                     BEAUTY                            2    0.0            0   
                     BEVERAGES                         3    0.0            0   
                     BOOKS                             4    0.0            0   
...                                                  ...    ...          ...   
2017-08-31 9         POULTRY                     3029395    NaN            1   
                     PREPARED FOODS              3029396    NaN            0   
                     PRODUCE                     3029397    NaN            1   
                     SCHOOL AND OFFICE SUPPLIES  3029398    NaN            9   
                     SEAFOOD                     3029399    NaN            0   

                                                   oil  is_holiday  year  \
date       store_nbr family                                                
2013-01-01 1         AUTOMOTIVE                  93.14        True  2013   
                     BABY CARE                   93.14        True  2013   
                     BEAUTY                      93.14        True  2013   
                     BEVERAGES                   93.14        True  2013   
                     BOOKS                       93.14        True  2013   
...                                                ...         ...   ...   
2017-08-31 9         POULTRY                     47.26       False  2017   
                     PREPARED FOODS              47.26       False  2017   
                     PRODUCE                     47.26       False  2017   
                     SCHOOL AND OFFICE SUPPLIES  47.26       False  2017   
                     SEAFOOD                     47.26       False  2017   

                                                month day_of_week  
date       store_nbr family                                        
2013-01-01 1         AUTOMOTIVE                     1           1  
                     BABY CARE                      1           1  
                     BEAUTY                         1           1  
                     BEVERAGES                      1           1  
                     BOOKS                          1           1  
...                                               ...         ...  
2017-08-31 9         POULTRY                        8           3  
                     PREPARED FOODS                 8           3  
                     PRODUCE                        8           3  
                     SCHOOL AND OFFICE SUPPLIES     8           3  
                     SEAFOOD                        8           3  

[3029400 rows x 8 columns]

In [7]:
def make_multistep_target(ts, steps, reverse=False):
    shifts = reversed(range(steps)) if reverse else range(steps)
    return pd.concat({f'y_step_{i + 1}': ts.shift(-i-1) for i in shifts}, axis=1)


In [8]:
X_START = "2014-01-01"
X_END = "2017-07-30"  

TRAIN_START = "2014-01-01"
TRAIN_END = "2017-07-29"

TEST_DAY = "2017-07-30"
TEST_DAY_PLUS = "2017-07-31"

QUERY_START = "2017-07-31"
QUERY_END = "2017-08-15"
QUERY_DAY = "2017-08-16"
N_STEP_PREDICTION = 16

In [9]:
def make_mw_in_groups(
    df: pd.DataFrame,
    groupby: List[str] = [],
    column: str = "",
    window: List[int] | int = 30,
    center: List[bool] | bool  = False,
    min_periods: List[int] | int  = 1,
    aggregator: List[Literal["mean","sum","median","std","var"]] | Literal["mean","sum","median","std","var"] = "mean",
) -> pd.DataFrame:
     
    if isinstance(window,int):
        window = [window]
        
    window = list(filter(lambda x : x != 0, window))
    if len(window) == 0 :
        raise ValueError("Window value must be non-zero!")
    if isinstance(center,bool):
        center = [center] * len(window)
    if isinstance(min_periods,int):
        min_periods = [min_periods]*len(window)
    if isinstance(aggregator,str):
        aggregator = [aggregator]*len(window)
        
    
    def create_mw_columns(group):
        ma_group = pd.DataFrame(index=group.index)
        for index, val in enumerate(window):
            type_name = "lag" if val > 0 else "lead"
            if val < 0 :
                ma_group[f"{column}_{type_name}_{aggregator[index]}_{-val}"] = group[column].rolling(window=-val, center=center[index], min_periods=min_periods[index]).aggregate(aggregator[index]).shift(val)
            else:
                ma_group[f"{column}_{type_name}_{aggregator[index]}_{val}"] = group[column].shift(1).rolling(window=val, center=center[index], min_periods=min_periods[index]).aggregate(aggregator[index])
        
        return ma_group
    
    return cast(pd.DataFrame,df.reset_index(groupby).groupby(groupby, observed=True).apply(create_mw_columns, include_groups=False).reset_index(groupby).set_index(groupby, append=True).sort_index())
    

    
# oil_ma = make_mw_in_groups(
#     df = mdata,
#     groupby = secondary_index,
#     column = "oil",
#     window = [16,-2],
#     center = False,
#     min_periods = 1,
#     aggregator = "mean",
# )

# display(oil_ma)

In [10]:
skip = False
if skip is False:

    output = pd.DataFrame()
    demo_test = False
    limit = 10000
    cnt = 0
    for family in unique_families[:limit]:
        for store in unique_stores[:limit]:
            if cnt % 100 == 0:
                print(f"Processing {cnt+1} out of {len(unique_families)*len(unique_stores)}")
            cnt += 1
            group = data_final[(data_final.index.get_level_values("family") == family) & (data_final.index.get_level_values("store_nbr") == store)]
            group.loc[:,["sales_lag_1"]] = group["sales"].shift(1)
            group.loc[:,["sales_lag_2"]] = group["sales"].shift(2)
            group.loc[:,["sales_lag_3"]] = group["sales"].shift(3)
            group.loc[:,["sales_lag_ma_7"]]= group["sales"].shift(1).rolling(window=7, center=False, min_periods=1).mean()
            group.loc[:,["sales_lag_ma_14"]] = group["sales"].shift(1).rolling(window=14, center=False, min_periods=1).mean()
            group.loc[:,["sales_lag_ma_21"]] = group["sales"].shift(1).rolling(window=21, center=False, min_periods=1).mean()
            group.loc[:,["oil"]] = group["oil"].ffill().bfill()
            group.loc[:,["is_year_end"]] = group.index.get_level_values("date").month >=11
            group.loc[:,["quarter"]] = group.index.get_level_values("date").quarter
            group.loc[:,[f"oil_lead_ma_{N_STEP_PREDICTION}"]] = group["oil"].rolling(window=N_STEP_PREDICTION, center=False, min_periods=1).mean().shift(-16)
#             group.loc[:,[f"is_holiday_lead_ms_{N_STEP_PREDICTION}"]] = group["is_holiday"].rolling(window=N_STEP_PREDICTION,center=False,min_periods=1).sum().shift(-16)
#             group.loc[:,[f"onpromotion_lead_ms_{N_STEP_PREDICTION}"]] = group["onpromotion"].rolling(window=N_STEP_PREDICTION,center=False, min_periods=1).sum().shift(-16)
            dowm = make_mw_in_groups(group.set_index("day_of_week", append=True), ["store_nbr","family","day_of_week"], "sales", 10, False, 1, "mean").reset_index("day_of_week", drop=True)
            group = group.join(dowm)
            
            X = pd.get_dummies(group.drop(columns=["id","sales"]), drop_first=True)
            y = make_multistep_target(group["sales"],steps=N_STEP_PREDICTION)

            if demo_test is False:
                X_final, y_final = X.loc[X_START:X_END], y.loc[X_START:X_END]
                X_query = X.loc[QUERY_END:QUERY_END]

                model = RegressorChain(XGBRegressor())
                model = model.fit(X_final, y_final)
                y_pred = model.predict(X_query)
                y_pred_df = pd.DataFrame({
                    "store_nbr":[store]*N_STEP_PREDICTION,
                    "family":[family]*N_STEP_PREDICTION, 
                    "date":pd.period_range(QUERY_DAY,periods=N_STEP_PREDICTION), 
                    "sales":y_pred[0]
                }).set_index(main_index)

                y_pred_df["id"] = group.loc[QUERY_DAY:]["id"]
                output = pd.concat([output, y_pred_df], axis=0)

            else:

                X_train, y_train = X.loc[TRAIN_START:TRAIN_END], y.loc[TRAIN_START:TRAIN_END]
                X_test, y_test = X.loc[TEST_DAY:TEST_DAY], y.loc[TEST_DAY:TEST_DAY]

                model = RegressorChain(XGBRegressor())
                model = model.fit(X_train, y_train)
                y_pred = model.predict(X_test)
                y_pred_df = pd.DataFrame({
                    "store_nbr":[store]*N_STEP_PREDICTION,
                    "family":[family]*N_STEP_PREDICTION, 
                    "date":pd.period_range(TEST_DAY_PLUS,periods=N_STEP_PREDICTION), 
                    "sales":y_pred[0]
                }).set_index(main_index)

                y_pred_df["id"] = group.loc[QUERY_DAY:]["id"]
                output = pd.concat([output, y_pred_df], axis=0)



    if demo_test is False:
        output["sales"] = output["sales"].clip(lower=0)
        output = output.reindex(columns=["id","sales"])
        output.to_csv("singular_model_submission.csv", index=False)
    else:
        output["sales"] = output["sales"].clip(lower=0)
        output = output.reindex(columns=["id","sales"])
        output.to_csv("testing_rmse.csv", index=False)
#         rmse = mean_squared_error(output.sort_index()["sales"], data_final["sales"])
#         print(f"RMSE: {rmse}")

Processing 1 out of 1782
Processing 101 out of 1782
Processing 201 out of 1782
Processing 301 out of 1782
Processing 401 out of 1782
Processing 501 out of 1782
Processing 601 out of 1782
Processing 701 out of 1782
Processing 801 out of 1782
Processing 901 out of 1782
Processing 1001 out of 1782
Processing 1101 out of 1782
Processing 1201 out of 1782
Processing 1301 out of 1782
Processing 1401 out of 1782
Processing 1501 out of 1782
Processing 1601 out of 1782
Processing 1701 out of 1782


In [11]:
skipMulti = True

NOW WORKING ON MAKING MULTIPROBLEM MODEL

In [12]:
def make_shift_in_groups(
    df: pd.DataFrame,
    groupby: List[str] = [],
    column: str = "",
    shift: int = 0,
    shift_list: List[int] = [],
    name: str | None = None,
) -> pd.DataFrame:
    shift_list = list(filter(lambda el: el != 0, [*shift_list, shift]))

    if len(shift_list) == 0:
        raise ValueError(
            "Shift value must be different than 0 or valid shift_list must be provided"
        )
 
    if name is None:
        name = column

    def create_lagged_columns(group):
        lagged_group = pd.DataFrame(index=group.index)
        for shift in shift_list:

            lagged_group[f"{name}_{'lead' if shift < 0 else 'lag'}_{abs(shift)}"] = (
                group[column].shift(shift)
            )

        return lagged_group

    lagged_df = cast(
        pd.DataFrame,
        df.reset_index(groupby)
        .groupby(groupby, observed=True)
        .apply(create_lagged_columns, include_groups=False).reset_index(groupby).set_index(groupby, append=True).sort_index(),
    )

    return lagged_df

In [13]:
if skipMulti is False:
    display(mdata)
    display(make_shift_in_groups(
            df = mdata, 
            groupby = secondary_index, 
            column = "sales",
            shift_list = [1,2]
        ).loc["2014"])
    display(make_mw_in_groups(
            df = mdata,
            groupby = secondary_index,
            column = "sales",
            window = [7,30],
            aggregator = "mean"
        ).loc["2014"])

In [14]:
# mdata is the copy of dataframe that willbe used for this model
if skipMulti is False:
        grouped_lags_leads : List[pd.DataFrame] = [

            make_shift_in_groups(
                df = mdata, 
                groupby = secondary_index, 
                column = "sales",
                shift_list = [1,2]
            ),
            make_mw_in_groups(
                df = mdata,
                groupby = secondary_index,
                column = "sales",
                window = [7,30],
                aggregator = "mean"
            ),
            make_mw_in_groups(
                df = mdata,
                groupby = secondary_index,
                column = "oil",
                window = [16,-16],
                aggregator = "mean"
            ),
            make_mw_in_groups(
                df = mdata,
                groupby = secondary_index,
                column = "is_holiday",
                window = -16,
                aggregator = "sum"
            ),
            make_mw_in_groups(
                df = mdata,
                groupby = secondary_index,
                column = "onpromotion",
                window = -16,
                aggregator = "sum"
            ),
        ]

        mdata_enhanced = mdata.join(grouped_lags_leads)

In [15]:
if skipMulti is False:
    mdata_enhanced["family_cat"] = mdata_enhanced.index.get_level_values("family")
    display(mdata_enhanced.columns)
    display(mdata_enhanced.select_dtypes("category").columns)

In [16]:
if skipMulti is False:
    X_prepared = pd.get_dummies(mdata_enhanced.drop(columns=["id","sales"]), drop_first=True)
    y_prepared = make_shift_in_groups(
        df = mdata_enhanced["sales"],
        groupby = secondary_index,
        column = "sales",
        shift_list = [-i for i in range(1,17)],
)

In [17]:
if skipMulti is False:
    X_train3, y_train3 = X_prepared.loc[X_START:X_END], y_prepared.loc[X_START:X_END]
    X_query3 = X_prepared.loc[QUERY_END:QUERY_END]


In [18]:
if skipMulti is False:
    display(X_train3)

In [19]:
if skipMulti is False:
    model = RegressorChain(XGBRegressor())
    model = model.fit(X_train3, y_train3)
    y_demo_pred3 = model.predict(X_query3)


In [20]:
if skipMulti is False:
    pred_df3 = pd.DataFrame(y_demo_pred3, index=X_query3.index, columns=y_prepared.columns).reset_index("date",drop=True).rename(columns={
        f"sales_lead_{i+1}" : pd.period_range(QUERY_END, periods=17)[i+1] for i in range(16)
    })
    display(pred_df3)

In [21]:
if skipMulti is False:
    def make_output(df)->pd.DataFrame:

        def create_output_rows(group):
            temp = group.tail(1).squeeze().rename("sales").to_frame()
            temp.index.name = "date"

            return temp

        output_df = cast(pd.DataFrame, df.reset_index().groupby(secondary_index, observed=True).apply(create_output_rows, include_groups=False).reset_index(secondary_index).set_index(secondary_index, append=True).sort_index())
        output_df_combined = output_df.join(mdata_enhanced["id"]).reindex(columns=["id","sales"])
        output_df_combined["sales"] = output_df_combined["sales"].clip(lower=0.0)

        return output_df_combined

In [22]:
if skipMulti is False:
    make_output(pred_df3).to_csv("multi_submission2.csv",index=False)